In [1]:
# Load Packages
from __future__ import unicode_literals, print_function

import plac #  wrapper over argparse
import random
from pathlib import Path
import spacy
from tqdm import tqdm, tqdm_notebook # loading bar 
import pandas as pd
import re
from pprint import pprint
from nltk.tokenize import sent_tokenize
import numpy as np
import unicodedata
from spacy.util import minibatch, compounding

In [2]:
# Função para substituir acentos

def strip_accents(text):
    try:
        text = unicode(text, 'utf-8')
    except (TypeError, NameError): # unicode is a default on python 3 
        pass
    text = unicodedata.normalize('NFD', text)
    text = text.encode('ascii', 'ignore')
    text = text.decode("utf-8")
    
    return str(text)

# Abrindo e gerando Datasets

### Entidade logradouro

In [ ]:
# Obtendo dados crus de endereço

dataset = pd.read_csv("201906AGENCIAS.CSV")
dset = dataset.iloc[:-2,4:10].values  # tipo nparray object
pd_dset = pd.DataFrame(dset) # caso queira visualizar

pd_dset['Join'] = pd_dset[pd_dset.columns[0:]].apply(
    lambda x: ';'.join(x.dropna().astype(str)),
    axis=1
)
pd_dset.head(5)

In [ ]:
# Tratando endereço completo do DataFrame
dset = np.array(pd_dset)
end_lista = []

for i in range(len(dset)):
    str_raw = dset[i][6]
    str_tratada = re.sub(r'[ ]{2,}', "",str_raw) # Tirando espaços excedentes no final do endereço
    str_tratada = re.sub(r'[;]{1,}', "; ", str_tratada) # Para complementos vazios, para não ter 2 ";"
    str_tratada = str_tratada.lower()
    str_tratada = str_tratada.replace(",", " ")
    str_tratada = str_tratada.replace(".", " ")
    str_tratada = strip_accents(str_tratada)
    str_tratada = re.sub(r'[ ]{2,}', " ", str_tratada)

    end_lista.append(str_tratada)

end_lista[:3]

In [ ]:
# Lógica para extrair posição do logradouro:
# Dividir a String inteira por ";", pegar o len do primeiro split
# len de LOGRA é de 0 até len do primeiro split

print(end_lista[0])
split = end_lista[0].split(";")
print(split)
print(split[0])
print(len(split[0]))

In [ ]:
# Extraindo iob com entidade apenas de logradouro e fazendo mais tratamentos de texto

iob = []

for i in range(len(end_lista)):
    split = end_lista[i].split(";")
    len_logra = len(split[0])
    iob_dict = {"entities": [(0, len_logra-1,'LOGRA')]}
    end_lista[i] = end_lista[i].replace(";", " ")
    end_lista[i] = end_lista[i].replace("  ", " ")
    end_lista[i] = end_lista[i].replace(",", " ")
    end_lista[i] = re.sub(r'^[ ]+', "", end_lista[i])
    
    tupla = (end_lista[i], iob_dict)
    iob.append(tupla)

FULL_DATA = iob

In [ ]:
print(FULL_DATA[0])
print(FULL_DATA[1])
print(FULL_DATA[2])

In [ ]:
# Criação da base de teste e treinamento

n_test= 0.1 # Porcentagem para base de teste
test_n = round(len(FULL_DATA) * n_test)

# Divisao em Train Test Val

def gerador_bases(dataset, n):
    indices_random = random.sample(range(0,len(dataset)-1), n)
    base_teste_n = []
    base_treinamento_n = []
    
    for i in range(n):
        base_teste_n.append(dataset[indices_random[i]])

    for j in range(len(dataset)):
        if(j not in indices_random):
            base_treinamento_n.append(dataset[j])
            
    return base_teste_n, base_treinamento_n


base_teste, base_treinamento = gerador_bases(FULL_DATA, test_n)

random.shuffle(base_treinamento)
random.shuffle(base_teste)

print("Treinamento: " + str(len(base_treinamento)), "\nTeste: " + str(len(base_teste)), "\nTotal: " + str(len(FULL_DATA)))

In [ ]:
for i in range(5):
    print(base_treinamento[i])

# Carregando o modelo

In [ ]:
# Define our variables

model = None
output_dir=Path(".")
n_iter= 100 # número de épocas
batch_size = 32

In [ ]:
# Setting up the pipeline and entity recognizer.
if model is not None:
    nlp = spacy.load(model)  # load existing spacy model
    print("Loaded model '%s'" % model)
else:
    lan = 'pt'
    nlp = spacy.blank(lan)  # create blank Language class
    print("Created blank '%s' model" % lan)
    
if 'ner' not in nlp.pipe_names:
    ner = nlp.create_pipe('ner')
    nlp.add_pipe(ner)
    print('Added new NER')
else:
    ner = nlp.get_pipe('ner')
    print('Got an old NER')

In [ ]:
# create the built-in pipeline components and add them to the pipeline
    # nlp.create_pipe works for built-ins that are registered with spaCy
if 'ner' not in nlp.pipe_names:
    ner = nlp.create_pipe('ner')
    nlp.add_pipe(ner, last=True)
# otherwise, get it so we can add labels
else:
    ner = nlp.get_pipe('ner')

## Treinamento

In [ ]:
print("Batch size: ", batch_size)
print("Épocas: ", n_iter)
print()

# add labels
for _, annotations in base_treinamento:
    for ent in annotations.get('entities'):
        ner.add_label(ent[2])


# get names of other pipes to disable them during training
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
with nlp.disable_pipes(*other_pipes):  # only train NER
    optimizer = nlp.begin_training()
    for itn in range(n_iter):
        random.shuffle(base_treinamento)
        losses = {}
        batches = minibatch(base_treinamento, size=batch_size)
        
        for batch in batches:
            texts, annotations = zip(*batch)
            try:
                nlp.update(texts, annotations, sgd=optimizer, drop=0.2,
                       losses=losses)
            except:
                pass     
        
        print(itn+1, ' Losses', losses)        

## Teste

In [ ]:
# Com base de treinamento
for text, _ in base_teste:
    doc = nlp(text)
    print('Entities', [(ent.text, ent.label_) for ent in doc.ents])
    print('Tokens', [(t.text, t.ent_type_, t.ent_iob) for t in doc])
    print()

In [ ]:
# Função para tratamento de texto de entrada

def regex(str_):
    ph_tratada = str_.replace("\"", "")
    ph_tratada = ph_tratada.replace("-", " ")
    ph_tratada = ph_tratada.replace("|", "")
    ph_tratada = ph_tratada.replace(",", "")
    ph_tratada = ph_tratada.replace(":", "")
    ph_tratada = ph_tratada.replace(".", "")
    ph_tratada = ph_tratada.replace("  ", "")
    ph_tratada = ph_tratada.lower()
    ph_tratada = re.sub(r'[\n]+', " ", ph_tratada)
    ph_tratada = re.sub(r'^[ ]+', "", ph_tratada)

    ph_tratada = strip_accents(ph_tratada
                              )    
    return ph_tratada

In [ ]:
# Com texto IPTU

ph ="""
        "PREFEITURA DE Certidão de Dados Cadastrais do Imóvel - IPTU 2017\n\nSÃO PAULO\n\nFAZENDA",
        "Cadastro do Imóvel: 067.061.0048-1",
        "| Local do Imóvel:",
        "| AVJACANA, 764- AP 104 E VG",
        "BLOCO 1 UP LIFE SAO PAULO CEP 02273-001\nImóvel localizado na 22 Subdivisão da Zona Urbana",
        "| Endereço para entrega da notificação:",
        "| AVJACANA, 764- AP 104 E VG",
        "| BLOCO1 UPLIFE SAO PAULO CEP 02273-001",
        "| Contribuinte(s):",
        "| AGUA DAS FLORES EMPREENDIMENTOS IMOB LTDA",
        "| Dados cadastrais do terreno:",
        "| Área incorporada (m?): 1.669 Testada (m): 0,00",
        "| Área não incorporada (m?): 0 Fração ideal: 0,0024",
        "| Área total (m?): 7.669",
        "| Dados cadastrais da construção:",
        "| Área construída (m?):\n\n88\n\nPadrão da construção:\n\n2-C",
        "Área ocupada pela construção (m?): 5.134 Uso: residência",
        "| Ano da construção corrigido: 2016",
        "| Valores de m? (R$):",
        "| - de terreno: 700,00",
        "| - da construção: 1.508,00",
        "| Valores para fins de cálculo do IPTU (R$):",
        "| - da área incorporada: 23.675,00",
        "- da área não incorporada: 0,00",
        "| - da construção: 131.377,00",
        "|| Base de cálculo do IPTU: 155.052,00",
        "Ressalvado o direito da Fazenda Pública do Município de São Paulo atualizar os dados constantes do Cadastro\nImobiliário Fiscal, apurados ou verificados a qualquer tempo, inclusive em relação ao exercício abrangido por\nesta certidão, a Secretaria Municipal da Fazenda CERTIFICA que os dados cadastrais acima foram utilizados no\nlançamento do Imposto Predial e Territorial Urbano do imóvel do exercício de 2017.",
        "Certidão expedida via Internet - Portaria SF nº 008/2004, de 28/01/2004.",
        "A autenticidade desta certidão poderá ser confirmada, até o dia 05/02/2018, em",
        "http://www .prefeitura.sp.gov.br/cidade/secretarias/financas/servicos/certidoes/",
        "| Data de Emissão: 07/11/2017",
        "| Número do Documento: 2.2017.001599486-1",
        "|| Solicitante: PAULO AFONSO DECICINO (CPF 892.407.408-30)"
    """      


In [ ]:
ph_tratada = regex(ph)
print(ph_tratada)

In [ ]:
doc_ph = nlp(ph_tratada)
print('Entities', [(ent.text, ent.label_) for ent in doc_ph.ents])

In [ ]:
# Com texto qualquer

phrase = regex("rua alameda gravatá quadra 301 conjunto 5 lote 08")

doc = nlp(phrase)
print('Entities', [(ent.text, ent.label_) for ent in doc.ents])
print('Tokens', [(t.text, t.ent_type_, t.ent_iob) for t in doc])